In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np

In [2]:
data = pd.read_parquet("/Users/kdugg/Documents/2003Q2").drop('LOAN_IDENTIFIER', axis = 1).values
mean = data.mean(axis=0)
data -= mean
std = data.std(axis=0)
data /= std

In [3]:
data

array([[-1.28428642, -0.06940043, -1.29501426, ..., -1.04426606,
        -0.66298563, -0.63451084],
       [-1.26078936, -0.40919923, -1.27150596, ..., -1.04167218,
        -0.66298563, -0.63451084],
       [-1.23729231, -1.02933204, -1.24799766, ..., -1.03906642,
        -0.66298563, -0.63451084],
       ...,
       [ 0.80695139,  1.13238798,  0.79722429, ..., -0.26465823,
        -0.44920583, -0.42302636],
       [ 0.83044845,  0.71863297,  0.82073258, ..., -0.27644552,
        -0.44920583, -0.42302636],
       [ 0.8539455 ,  0.67765723,  0.84424088, ..., -0.28738135,
        -0.43755717, -0.42024012]])

In [4]:
def generator(batch_size, qtrs):
    count = 0
    qtri = 0
    raw = pd.read_parquet("/Users/kdugg/Documents/"+qtrs[qtri]).drop('LOAN_IDENTIFIER', axis = 1).values
    mean = raw.mean(axis=0)
    raw -= mean
    std = raw.std(axis=0)
    raw /= std
    data = np.full((batch_size, 216, 8), -1)
    while True:
        for i in range(1, len(raw)):
            if i>0 and raw[i][0] != raw[i-1][0] + 1:
                count += 1
            if count == batch_size:
                yield data
                data = np.full((batch_size, 216, 8), -1)
                count = 0
            day = raw[i][0]
            data[count][day] = raw[i][1:]
        qtri += 1
        if qtri == len(qtrs):
            qtri = 0
        raw = pd.read_parquet("/Users/kdugg/Documents/"+qtrs[qtri]).drop('LOAN_IDENTIFIER', axis = 1).values
        mean = raw.mean(axis=0)
        raw -= mean
        std = raw.std(axis=0)
        raw /= std

In [ ]:
cur = raw[0][0]
count = 0
for i in range(len(raw)):
    if raw[i][0] != cur:
        count += 1
        cur = raw[i][0]
    day = raw[i][1]
    pos = 12*int(day[-2:])+int(day[:2])
    data[count][pos] = raw[i][2:]

In [8]:
perf = perf.merge(popinc, left_on = ['zip', 'date'], right_on = ['ZIP_CODE_SHORT', 'DATE']).drop('ZIP_CODE_SHORT', axis = 1).drop('DATE', axis = 1)
popinc = None
perf['msa'] = perf['msa'].astype('str')
perf = perf.merge(unemploy, left_on = ['msa', 'date'], right_on = ['METROPOLITAN_STATISTICAL_AREA', 'DATE'])
unemploy = None
perf = perf.drop('METROPOLITAN_STATISTICAL_AREA', axis = 1).drop('DATE', axis = 1).drop('UNEMPLOYMENT', axis = 1).drop('EMPLOYMENT', axis = 1).drop('LABOR_FORCE', axis = 1)
perf = perf.sort_values(['order', 'age'])

In [9]:
perf = perf.drop('order', axis = 1).reset_index(drop = True)

In [10]:
perf

,id,ltv,credit_score,zip,date,interest,upb,age,mature,msa,PERSONAL_INCOME,POPULATION,UNEMPLOYMENT_RATE
0,981956553826,80.0,720.0,850,12/01/2002,6.125,104000.00,1,359.0,38060,9.700816e+07,3.255388e+06,5.4
1,981956553826,80.0,720.0,850,01/01/2003,6.125,104000.00,2,358.0,38060,1.023343e+08,3.328468e+06,5.8
2,981956553826,80.0,720.0,850,02/01/2003,6.125,104000.00,3,357.0,38060,1.023343e+08,3.328468e+06,5.5
3,981956553826,80.0,720.0,850,03/01/2003,6.125,104000.00,4,356.0,38060,1.023343e+08,3.328468e+06,5.4
4,981956553826,80.0,720.0,850,04/01/2003,6.125,104000.00,5,355.0,38060,1.023343e+08,3.328468e+06,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61225791,818720508026,72.0,765.0,600,02/01/2003,5.500,227000.00,3,176.0,16980,8.337720e+07,2.151851e+06,7.1
61225792,818720508026,72.0,765.0,600,03/01/2003,5.500,227000.00,4,176.0,16980,8.337720e+07,2.151851e+06,7.0
61225793,818720508026,72.0,765.0,600,04/01/2003,5.500,221680.69,5,173.0,16980,8.337720e+07,2.151851e+06,6.6
61225794,818720508026,72.0,765.0,600,05/01/2003,5.500,221680.69,6,173.0,16980,8.337720e+07,2.151851e+06,6.6


In [13]:
def generator(batch_size, raw):
    cur = raw[0][0]
    while True:
        data = np.zeros((batch_size, 240, 11))
        count = 0
        for i in range(len(raw)):
            if raw[i][0] != cur:
                count += 1
                cur = raw[i][0]
            if count == batch_size:
                yield data
            day = raw[i][4]
            pos = 12*int(day[-2:])+int(day[:2])-1
            data[count][pos] = raw[i][2:]

In [18]:
perf[perf['id'] == 813252262387]

,id,ltv,credit_score,zip,date,interest,upb,age,mature,msa,PERSONAL_INCOME,POPULATION,UNEMPLOYMENT_RATE
537,813252262387,63.0,788.0,627,12/01/2002,5.625,136000.0,-1,180.0,44100,6252967.0,190756.0,4.6
538,813252262387,63.0,788.0,627,01/01/2003,5.625,136000.0,0,180.0,44100,6250612.0,191339.0,5.6
539,813252262387,63.0,788.0,627,02/01/2003,5.625,136000.0,1,179.0,44100,6250612.0,191339.0,5.7
540,813252262387,63.0,788.0,627,03/01/2003,5.625,136000.0,2,178.0,44100,6250612.0,191339.0,5.3
541,813252262387,63.0,788.0,627,04/01/2003,5.625,136000.0,3,177.0,44100,6250612.0,191339.0,4.5
542,813252262387,63.0,788.0,627,05/01/2003,5.625,136000.0,4,176.0,44100,6250612.0,191339.0,4.8
543,813252262387,63.0,788.0,627,06/01/2003,5.625,132996.7,5,175.0,44100,6250612.0,191339.0,5.8
544,813252262387,63.0,788.0,627,07/01/2003,5.625,132996.7,6,0.0,44100,6250612.0,191339.0,5.6


In [16]:
acq[acq[0]==813252262387]

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
989600,813252262387,R,OTHER,5.625,136000,180,12/2002,02/2003,63,63.0,...,SF,4,I,IL,627,NaN,FRM,799.0,NaN,N


In [11]:
df924204458121 = perf[perf['id']==924204458121]

In [12]:
df924204458121

,id,date,interest,upb,age,mature,msa
65624220,924204458121,11/01/2002,6.5,NaN,1,359.0,33460
65624221,924204458121,12/01/2002,6.5,NaN,2,358.0,33460
65624222,924204458121,01/01/2003,6.5,NaN,3,357.0,33460
65624223,924204458121,02/01/2003,6.5,NaN,4,356.0,33460
65624224,924204458121,03/01/2003,6.5,NaN,5,355.0,33460
...,...,...,...,...,...,...,...
65624413,924204458121,12/01/2018,4.5,223581.97,194,NaN,33460
65624414,924204458121,01/01/2019,4.5,223171.21,195,NaN,33460
65624415,924204458121,02/01/2019,4.5,222759.08,196,NaN,33460
65624416,924204458121,03/01/2019,4.5,222345.45,197,NaN,33460


In [14]:
df924204458121[df924204458121['age']==182.0]

,id,date,interest,upb,age,mature,msa
65624401,924204458121,12/01/2017,4.0,229699.78,182,NaN,33460


In [1]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import gzip
import s3fs
import os
s3 = s3fs.S3FileSystem()
qtrs = ["20{}Q{}".format(str(i).zfill(2), j) for i in range(3, 18) for j in range(1, 5)]
for qtr in qtrs:
    raw = pq.ParquetDataset('s3://loan-performance-data/shuffled_data/qtr='+qtr, filesystem=s3).read_pandas().to_pandas()
    num = len(raw['LOAN_IDENTIFIER'].unique())
    raw['ADJUSTED_MONTHS_TO_MATURITY'] = raw['ADJUSTED_MONTHS_TO_MATURITY'].apply(lambda x: 0 if x==0 else 1)
    ids = raw['LOAN_IDENTIFIER'].to_numpy()
    mature = raw['ADJUSTED_MONTHS_TO_MATURITY'].to_numpy()
    raw = raw.drop('LOAN_IDENTIFIER', axis = 1).drop('MONTHLY_REPORTING',axis = 1).drop('ADJUSTED_MONTHS_TO_MATURITY', axis = 1).values
    mean = raw.mean(axis=0)
    raw -= mean
    std = raw.std(axis=0)
    raw /= std
    data = np.zeros((num, 216, 8))
    count = 0
    j = 0
    for i, row in enumerate(raw):
        if i>0 and ids[i] != ids[i-1]:
            count += 1
            j = 0
        data[count][j] = np.nan_to_num(np.append(row, mature[i]))
        j += 1
    f = gzip.GzipFile(qtr+'.npy.gz', "w")
    np.save(f, data)
    f.close()
    os.system('aws s3api put-object --bucket loan-performance-data --key numpy2/'+qtr+'.npy.gz --body '+qtr+'.npy.gz')
    os.system('rm '+qtr+'.npy.gz')

In [1]:
import pandas as pd
import numpy as np
import os
qtrs= [1]
for qtr in qtrs:
    raw = pd.read_parquet('/Users/kdugg/Documents/2003Q2')
    num = len(raw['LOAN_IDENTIFIER'].unique())
    raw['ADJUSTED_MONTHS_TO_MATURITY'] = raw['ADJUSTED_MONTHS_TO_MATURITY'].apply(lambda x: 0 if x==0 else 1)
    ids = raw['LOAN_IDENTIFIER'].to_numpy()
    mature = raw['ADJUSTED_MONTHS_TO_MATURITY'].to_numpy()
    raw = raw.drop('LOAN_IDENTIFIER', axis = 1).drop('MONTHLY_REPORTING',axis = 1).drop('ADJUSTED_MONTHS_TO_MATURITY', axis = 1).values
    mean = raw.mean(axis=0)
    raw -= mean
    std = raw.std(axis=0)
    raw /= std
    data = np.zeros((num, 216, 8))
    count = 0
    j = 0
    for i, row in enumerate(raw):
        if i>0 and ids[i] != ids[i-1]:
            count += 1
            j = 0
        data[count][j] = np.append(row, mature[i])
        j += 1
    np.save('2002Q3', data)

In [2]:
np.argwhere(np.isnan(data))

array([], shape=(0, 3), dtype=int64)

In [1]:
import numpy as np
data = np.load('2002Q3.npy', mmap_mode = 'r')
for row in data[1181500]:
    print(row)

[-0.05940634 -1.22448936 -0.62831638 -0.98279661 -0.85750499 -0.58024009
 -0.55898433  1.        ]
[ 0.19794128 -1.20098107 -0.62831638 -0.89806675 -0.85018905 -0.58024009
 -0.55898433  1.        ]
[ 0.43030369 -1.17747277 -0.62831638 -0.68624208 -0.84332973 -0.58024009
 -0.55898433  1.        ]
[ 0.09050489 -1.15396447 -0.62831638 -0.81333688 -0.83719763 -0.58024009
 -0.55898433  1.        ]
[-0.52962792 -1.13045617 -0.62831638 -0.81333688 -0.83194928 -0.58024009
 -0.55898433  1.        ]
[-0.39920515 -1.10694787 -0.62831638 -0.85570182 -0.82617917 -0.58024009
 -0.55898433  1.        ]
[-0.21581373 -1.08343958 -0.62831638 -0.98279661 -0.819396   -0.58024009
 -0.55898433  1.        ]
[-0.21431462 -1.05993128 -0.62831638 -0.98279661 -0.81239895 -0.58024009
 -0.55898433  1.        ]
[-0.14585516 -1.03642298 -0.62831638 -1.06752648 -0.80602088 -0.58024009
 -0.55898433  1.        ]
[ 0.043033   -1.01291468 -0.62831638 -0.77097195 -0.79942438 -0.57178669
 -0.55724047  1.        ]
[ 0.120487

In [10]:
for row in data[100]:
    print(row)

[-0.25429095 -1.29501426  0.72950653 -0.43205249 -1.03051642 -0.68516663
 -0.67461429  1.        ]
[ 0.07401466 -1.27150596  0.72950653 -0.34732262 -1.02827283 -0.68516663
 -0.67461429  1.        ]
[ 0.32786435 -1.24799766  0.72950653 -0.00840316 -1.02578784 -0.68516663
 -0.67461429  1.        ]
[-0.07439747 -1.22448936  0.72950653 -0.26259276 -1.02385342 -0.68516663
 -0.67461429  0.        ]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0.

In [4]:
import pandas as pd
raw = pd.read_parquet('/Users/kdugg/Documents/2003Q2')
raw

,LOAN_IDENTIFIER,MONTHLY_REPORTING,DELTA_INTEREST,LOAN_AGE,ADJUSTED_MONTHS_TO_MATURITY,ORIGINAL_LOAN-TO-VALUE,UNEMPLOYMENT_RATE,HOUSING_PRICE,PERSONAL_INCOME,POPULATION
0,196072151802,41,0.2475,1.0,179.0,69.0,4.8,107644.333333,2.942143e+06,108356.800000
1,196072151802,42,-0.0925,2.0,178.0,69.0,4.9,108089.000000,2.942143e+06,108356.800000
2,196072151802,43,-0.7130,3.0,177.0,69.0,4.4,108535.703704,2.942143e+06,108356.800000
3,196072151802,44,-0.5825,4.0,176.0,69.0,3.9,109170.740741,2.942143e+06,108356.800000
4,196072151802,45,-0.3990,5.0,175.0,69.0,3.8,109859.333333,2.942143e+06,108356.800000
...,...,...,...,...,...,...,...,...,...,...
86710449,516174181688,128,1.4040,88.0,225.0,80.0,9.4,244688.426230,1.677437e+07,421606.483146
86710450,516174181688,129,1.5250,89.0,224.0,80.0,9.3,243190.360656,1.677437e+07,421606.483146
86710451,516174181688,130,1.4500,90.0,223.0,80.0,9.6,241292.016393,1.677437e+07,421606.483146
86710452,516174181688,131,1.0360,91.0,221.0,80.0,9.1,239271.327869,1.677437e+07,421606.483146
